In [ ]:
!git clone https://github.com/Octavio341/PruebaconGoogleColab.git

In [1]:
#[0]=================  Cuadros y Grafica Dividida ==========================
from bokeh.plotting import figure, show, output_notebook
from bokeh.io import push_notebook
import numpy as np
#[0]=================  Ventana y cuadros Y CARGAR ARCHIVOS ==========================
import ipywidgets as widgets
from IPython.display import display,clear_output,HTML
#[0]=================  CARGAR ARCHIVOS y Grafica Dividida ==========================
import os
from google.colab import files, drive
import hashlib
#[0]==================== Grafica Dividida =================================
import pandas as pd
from bokeh.models import Button as BokehButton,CustomJS,CheckboxGroup,ColumnDataSource, TableColumn, DataTable, DateFormatter,TabPanel, Tabs, Div,Select,TextInput,LabelSet,Slider, DatetimeTickFormatter
from bokeh.models.widgets import Button
from bokeh.palettes import Category10
from bokeh.layouts import column, row,gridplot#para la posiciones
from bokeh.embed import components
from bokeh.resources import CDN
import datetime
import ipywidgets as widgets
#para ejecutar scripts
import subprocess
#[1]==================== Grafica Dividida =================================
output_notebook()
#[A1]==================== Grafica Dividida =================================

Controles_zoom = widgets.Output()
estado = {}

#[4]==================== Grafica Dividida =================================
#mostramos la grafica

def actualizar_vista():
  nombres_archivos = []
  clear_output(wait=True)
  print("")
  herramientas=[
        "pan",
        "wheel_zoom",
        "box_zoom",
        "reset",
        "save",
        "lasso_select",
        "box_select",
        "tap",
        "zoom_in",
        "zoom_out",
        "help"
      ]
  p = figure(
    x_axis_label="Fecha",
    y_axis_label="Nivel del mar (mm)",
    x_axis_type="datetime",
    width=1300,
    height=600,
    tools=','.join(herramientas),
    toolbar_location="above",
    toolbar_sticky=False,
    margin=(5, 5, 5, 5)
  )
  #[2]==================== Grafica Dividida =================================
  # Ejemplo: Cambiar color de texto en una columna con HTMLTemplateFormatter
  css_personalizado = """
  .bk-data-table {
      background-color: #ffffff !important;
      color: #000000 !important;
      font-size: 16px !important;
      font-weight: normal !important;
      border: 9px solid #ddd !important;
      border-radius: 8px !important;
      font-family: "Segoe UI", Tahoma, Geneva, Verdana, sans-serif !important;
  }
  .bk-header {
      background-color: #f4d03f !important;
      color: #f9f9f9 !important;
      font-weight: bold !important;
      font-size: 18px !important;
  }
  .bk-data-table .slick-row:nth-child(even) {
      background-color: #595959 !important;
  }
  .bk-data-table .slick-row:nth-child(odd) {
      background-color: #595959 !important;
  }
  /* Selección destacada */
  .bk-data-table .slick-row.selected {
      background-color: #fffb91 !important;
      color: #000000 !important;
      font-weight: 900 !important;
      box-shadow: 0 0 10px 3px rgba(255, 255, 145, 0.6) !important;
      border: 2px solid #e1d700 !important;
      outline: none !important;
      filter: none !important;
      opacity: 1 !important;
      transition: background-color 0.3s ease, box-shadow 0.3s ease, border 0.3s ease;
  }
  .bk-data-table .slick-row:hover {
      background-color: rgba(255, 235, 59, 0.3) !important;
      color: #000000 !important;
      font-weight: 700 !important;
      box-shadow: inset 0 0 10px rgba(255, 235, 59, 0.4) !important;
      outline: 1px solid rgba(255, 235, 59, 0.7) !important;
      filter: none !important;
      opacity: 1 !important;
      cursor: pointer;
      transition: background-color 0.3s ease, box-shadow 0.3s ease, outline 0.3s ease;
  }
  .bk-data-table .slick-cell {
      border-right: 1px solid rgba(200, 200, 200, 0.5) !important;
  }
  .bk-data-table .slick-cell:last-child {
      border-right: none !important;
  }

  """

  #[3]==================== Grafica Dividida =================================
  #Uso de la tabla
  colores = Category10[10]
  source2 = []
  lineas = [] #para guardar las lineas

  pico_e = []
  valles_e = []
  nombres_archivos = []
  puntos = []
  all_labels = []
  sources_E = []
  nombre=[]
  for i, (df, archivo) in enumerate(zip(dfs, archivos_cargados)):
    labels_originales = [str(v) for v in df["Promedio"]]
    source2 = ColumnDataSource(data=dict(
        x=pd.to_datetime(df['Date']),
        y=df['Promedio'],
        labels=[""] * len(df["Promedio"]),
    ))
    # Inicializar la bolita en el primer punto

# ColumnDataSource de la bolita
    source_ball = ColumnDataSource(data=dict(
        x=[df['Date'].iloc[0]],
        y=[df['Promedio'].iloc[0]]
    ))
    source3 = source2
    ###################################################################### control de picos y valles
    from scipy.signal import find_peaks
    indices_picos, _ = find_peaks(source3.data["y"], distance=6)
    # Mínimos locales (bajamares)
    indices_valles, _ = find_peaks(-np.array(source3.data['y']), distance=6)

    df["pico"] = np.nan
    #####################################################################
    # --- Asignar valores a las columnas pico y valle dentro de source2 ---
    # Crear columnas 'pico' y 'valle' en source2 (inicialmente NaN)
    source3.data['pico'] = [np.nan] * len(source3.data['y'])
    source3.data['valle'] = [np.nan] * len(source3.data['y'])
    for ii in indices_picos:
        source3.data['pico'][ii] = source3.data['y'][ii]

    for ii in indices_valles:
        source3.data['valle'][ii] = source3.data['y'][ii]
    ######################################################################
      #crear la figura
    #p.line(source=source2, line_width=2, color="black")
    columns = [
        TableColumn(field="x", title="Fecha", formatter=DateFormatter(format="%Y-%m-%d"),width=500),
        TableColumn(field="y", title="Nivel"),
        TableColumn(field="x", title="Hora", formatter=DateFormatter(format="%H:%M:%S")),
    ]
    tabla = DataTable(source=source2, columns=columns, width=400, height=500,stylesheets=[css_personalizado])
    tablas.append(tabla)  # guardamos la tabla en la lista
    ###
    pico_1=p.scatter("x", "pico", source=source3, size=8, color="red", marker="triangle",legend_label="Picos (Pleamares)")

    # Para valles
    valle_1=p.scatter("x", "valle", source=source3, size=8, color="green", marker="inverted_triangle", legend_label="Valles (Bajamares)")
    ####
    #[]================================  MOVIMIENTO DE LOS PUNTOS  =======================================
    p.scatter('x', 'y', size=15, source=source_ball, color="red", legend_label="Bolita")
    # Conectar movimiento del mouse
    # CustomJS para mover la bolita y actualizar el Div
    # Callback para mover la bolita al punto más cercano
    callback_move = CustomJS(args=dict(source_ball=source_ball, source_data=source2), code="""
        const data = source_data.data;
        const ball = source_ball.data;
        const mouseX = cb_obj.x;

        // Encontrar el índice del punto más cercano en X
        let distances = data['x'].map(xi => Math.abs(xi - mouseX));
        let min_index = distances.indexOf(Math.min(...distances));

        // Mover la bolita
        ball['x'][0] = data['x'][min_index];
        ball['y'][0] = data['y'][min_index];

        source_ball.change.emit();
    """)

    #[1]================================  CASILLAS DE LAS LINEAS  =======================================
    linea2 = p.scatter('x', 'y', source=source2, legend_label=str(i), line_width=5, color=colores[i % len(colores)]) #las graficas con puntos
    linea1 = p.line(source=source2, line_width=2) #guardamos las lineas de nuestra grafica

    pico_e.append(pico_1)
    valles_e.append(valle_1)

    lineas.append(linea1)#guardamos este conjunto de lineas
    labels_set = LabelSet(x='x', y='y', text='labels', source=source2, text_color="red", text_font_size="8pt")
    p.add_layout(labels_set)
    nombre = os.path.basename(archivo)
    nombres_archivos.append(nombre)#guardamos el nombre del archivo
    puntos.append(linea2)
    sources_E.append(source2)
    all_labels.append(labels_originales)  # guardamos originales
  #[1]================================  CASILLAS DE LAS LINEAS  =======================================


  p.xaxis.axis_label = "Fecha"
  p.yaxis.axis_label = "Nivel del mar (mm)"
  p.title.text_font_size = "20pt"
  p.toolbar.autohide = True
  p.toolbar.logo = None
  p.legend.location = "top_right"
  p.title.text = f"Gráfica de ejemplo{nombres_archivos}"
  p.js_on_event('mousemove', callback_move)
  #[]================================  CASILLAS leyendas  =======================================
  #  checkbox_leyenda = CheckboxGroup(labels=["Mostrar Seno", "Mostrar Coseno", "Mostrar Seno+1"], active=[0,1,2])

  checkbox_leyenda = CheckboxGroup(labels=["activar Leyenda","Picos","valles"], active=[0,1,2])

  callback_legend_vis = CustomJS(args=dict(legend=p.legend[0], checkbox=checkbox_leyenda), code="""
      for (let i = 0; i < legend.items.length; i++) {
          let show = checkbox.active.includes(i);
          legend.items[i].visible = show;
      }
  """)

  checkbox_leyenda.js_on_change('active', callback_legend_vis)

  select_leyenda = Select(title="Posición leyenda", value="top_right", options=[
      "top_left", "top_center", "top_right",
      "center_left", "center", "center_right",
      "bottom_left", "bottom_center", "bottom_right",
      "top", "left", "center", "right", "bottom"
  ])

  callback_legend_pos = CustomJS(args=dict(legend=p.legend[0]), code="""
      legend.location = cb_obj.value;
  """)

  select_leyenda.js_on_change('value', callback_legend_pos)

  #[2]================================  CASILLAS DE LAS LINEAS  =======================================
  #ORIGINALES
  # Checkbox para controlar visibilidad
  checkbox_LiOriginal= CheckboxGroup(labels=nombres_archivos, active=list(range(len(nombres_archivos))))#creamos los chexkbox con forme a la cantidad de lineas

  # Callback JS para alternar visibilidad
  checkbox_LiOriginal.js_on_change("active", CustomJS(args=dict(lineas=lineas), code="""
      for (let i = 0; i < lineas.length; i++) {
          lineas[i].visible = this.active.includes(i);
      }
  """))
  #con nombre del archivo
  #checkbox_LiPuntos= CheckboxGroup(labels=nombres_archivos, active=list(range(len(nombres_archivos))))#creamos los chexkbox con forme a la cantidad de lineas
  checkbox_LiPuntos= CheckboxGroup(labels=[""] * len(nombres_archivos),active=list(range(len(nombres_archivos))))#creamos los chexkbox con forme a la cantidad de lineas
  #PUNTOS
  # Callback JS para alternar visibilidad
  checkbox_LiPuntos.js_on_change("active", CustomJS(args=dict(puntos=puntos), code="""
      for (let i = 0; i < puntos.length; i++) {
          puntos[i].visible = this.active.includes(i);
      }
  """))
  #TEXTO
  # JS callback para filtrar etiquetas
  checkbox_LiEtiqueta=CheckboxGroup(labels=[""] * len(nombres_archivos))
  callback_code = """
  for (let i = 0; i < sources_E.length; i++) {
      const source = sources_E[i];
      const original_labels = original_labels_list[i];
      source.data['labels'] = cb_obj.active.includes(i) ? original_labels : original_labels.map(() => "");
      source.change.emit();
  }
  """
  checkbox_LiEtiqueta.js_on_change("active", CustomJS(args=dict(
      sources_E=sources_E,
      original_labels_list=all_labels
  ), code=callback_code))

  #PICOS Y VALLES
  pico_1.visible = False
  valle_1.visible = False
  lineas_pv = pico_e + valles_e
  # Checkbox
  checkbox_pico_valle = CheckboxGroup(labels=["Mostrar Picos y Valles"])

  # Callback JS
  checkbox_pico_valle.js_on_change(
      "active",
      CustomJS(args=dict(lineas=lineas_pv), code="""
          const mostrar = this.active.includes(0); // solo una casilla
          for (let i = 0; i < lineas.length; i++) {
              lineas[i].visible = mostrar;
          }
      """)
  )
  #APLICAR RANGO CON CAJA DE TEXTO
  ##############################################################################
  #================================  filtros =======================================
  #filtros = column(,,)
  #############################   TABLA  ############################################################
  #[1]=====================  Nuevos widgets  ======================================================
  # ====== 2. Widgets ======
  # Supongamos que 'puntos' es un arreglo de GlyphRenderer (por ejemplo, resultado de p.circle(...))
  # 'colores' es un array de colores que corresponden a cada línea
  #checkbox_colores = CheckboxGroup(labels=["black","gray","orange","purple"], active=[0,1,2])
  checkbox_colores = CheckboxGroup(labels=["black","gray","orange","purple"])
  checkbox_1 = CheckboxGroup(labels=["Mostrar línea"], active=[0])
  text_input_1 = TextInput(value=p.title.text, title="Editar título:",width=900 )
  boton_1 = Button(label="Click aquí", button_type="success")

  # ====== 3. Callbacks JavaScript ======
  checkbox_colores.js_on_change(
      "active",
      CustomJS(args=dict(lineas=lineas, colores=["black","gray","orange","purple"]), code="""
          // Tomamos la primera casilla activa, o 'blue' si ninguna está activa
          let color = cb_obj.active.length > 0 ? colores[cb_obj.active[0]] : "blue";
          for (let i = 0; i < lineas.length; i++){
              lineas[i].glyph.line_color = color;
              lineas[i].change.emit();  // fuerza el redibujo
          }
      """)
  )
  text_input_1.js_on_change(
    "value",
    CustomJS(args=dict(figura=p), code="""
        figura.title.text = cb_obj.value;
    """)
  )


  p.title.text_font_size = "20pt"
  p.xaxis.axis_label_text_font_size = "16pt"
  p.yaxis.axis_label_text_font_size = "16pt"
  # Separar labels del eje X
  p.xaxis[0].major_label_standoff = 15
  p.xaxis[0].axis_label_standoff = 25

  #############################   guia  ############################################################

  # Formato inicial
  formatter = DatetimeTickFormatter(
      hours="%Y-%m-%a %d\n%H:%M:%S",
      days="%Y-%m-%a %d\n%H:%M:%S",
      months="%Y-%m-%a %d\n%H:%M:%S",
      years="%Y-%m-%a %d\n%H:%M:%S"
  )
  p.xaxis.formatter = formatter
  p.xaxis.major_label_orientation = 1.57  # vertical

  # Ajustes iniciales
  p.xaxis[0].ticker.desired_num_ticks = 5
  p.yaxis[0].ticker.desired_num_ticks = 5

  # Sliders para divisiones
  slider_x = Slider(start=2, end=30, value=5, step=1, title="Divisiones en eje X")
  slider_y = Slider(start=2, end=30, value=5, step=1, title="Divisiones en eje Y")

  # Checkboxes para formato
  checkboxes_guia = CheckboxGroup(labels=["Año", "Mes", "Día", "Hora"], active=[0,1,2,3])

  # Selector para orientación
  orientation_select = Select(title="Orientación etiquetas X", value="Vertical",
                              options=["Vertical", "Horizontal"])

  # JavaScript para sliders
  slider_x.js_on_change("value", CustomJS(args=dict(px=p.xaxis[0]), code="""
      px.ticker.desired_num_ticks = cb_obj.value;
  """))

  slider_y.js_on_change("value", CustomJS(args=dict(py=p.yaxis[0]), code="""
      py.ticker.desired_num_ticks = cb_obj.value;
  """))

  # JavaScript para checkboxes (formato dinámico con casos especiales)
  checkboxes_guia.js_on_change("active", CustomJS(args=dict(px=p.xaxis[0]), code="""
      const parts = {0: "%Y", 1: "%b", 2: "%a:%d", 3: "%H:%M:%S"};
      let active = cb_obj.active.sort();
      let fmt = active.map(i => parts[i]);

      let format_str = "";
      // Caso especial Día + Hora
      if (active.length === 2 && active.includes(2) && active.includes(3)) {
          format_str = "%a %d %H:%M:%S";
      }
      // Caso especial Año + Mes + Día
      else if (active.length === 3 && active.includes(0) && active.includes(1) && active.includes(2)) {
          format_str = "%Y %b %a:%d";
      }
      else {
          format_str = fmt.join(" ");
      }

      px.formatter.hours = format_str;
      px.formatter.days = format_str;
      px.formatter.months = format_str;
      px.formatter.years = format_str;
      px.change.emit();  // refrescar eje
  """))

  # JavaScript para orientación
  orientation_select.js_on_change("value", CustomJS(args=dict(px=p.xaxis[0]), code="""
      if (cb_obj.value === "Vertical") {
          px.major_label_orientation = 1.57;
      } else {
          px.major_label_orientation = 0;
      }
  """))
  #############################   TABLA  ############################################################
  tabla_layout =column(*tablas)
  i=0
  options = [f"Tabla {i+1}" for i in range(len(tablas))]
  select = Select(title="Seleccione tabla", value=options[0], options=options, width=200)

  # JS callback para cambiar visibilidad
  callback = CustomJS(args=dict(tabs=tablas, select=select), code="""
      const selected = select.value;
      for(let i=0; i<tabs.length; i++){
          if(selected === `Tabla ${i+1}`){
              tabs[i].visible = true;
          } else {
              tabs[i].visible = false;
          }
      }
  """)
  ##############################     TEXTOS TABAS         ##########################################

  # Texto en blanco y negrita
  Espacio = Div(
      text="<span style='color:black; font-weight:bold;'>     </span>",
      width=300, height=60
  )

  Linea_original = Div(
      text="<span style='color:black; font-weight:bold;'>LINEA DE DATOS </span>",
      width=200, height=30
  )
  punto_tab = Div(
      text="<span style='color:black; font-weight:bold;'>PUNTOS </span>",
      width=100, height=30
  )
  etiqueta_tab = Div(
      text="<span style='color:black; font-weight:bold;'>ETIQUETA</span>",
      width=100, height=30
  )
  colores_linea = Div(
      text="<span style='color:black; font-weight:bold;'> COLORES DE LINEA  </span>",
      width=100, height=30
  )
  creador_escala = Div(
      text="<span style='color:black; font-weight:bold;'> CREAR MAS ESCALAS EN LOS EJES </span>",
      width=300, height=30
  )
  Leyendas_grafica = Div(
      text="<span style='color:black; font-weight:bold;'> LEYENDAS DE LAS GRAFICAS  </span>",
      width=300, height=30
  )
  cambiar_nombre = Div(
      text="<span style='color:black; font-weight:bold;'> CAMBIAR EL NOMBRE DE LA GRAFICA  </span>",
      width=300, height=40
  )
  ############################################################################################
  #guia
  layout_guia = column(row(slider_x, slider_y),
    checkboxes_guia,
    orientation_select
  )
  #---
  #--- leyenda
  layout_leyenda = column(select_leyenda, checkbox_leyenda)


  select.js_on_change('value', callback)
  contenido_tabla = column(select, tabla_layout)
  #[2]=====================  Nuevos widgets  ======================================================
  # ====== 4. Contenedores ======
  # Columna de controles
  col_controles = column(Espacio,creador_escala,layout_guia,Leyendas_grafica, layout_leyenda,cambiar_nombre ,text_input_1, width=250)
  LinP = column(Espacio,Linea_original,checkbox_LiOriginal,colores_linea,checkbox_colores,width=250)
  LinP_2 = column(Espacio,punto_tab,checkbox_LiPuntos,width=80)
  LinP_3 = column(Espacio,etiqueta_tab,checkbox_LiEtiqueta,width=80)
  Tab_12 = row(LinP,LinP_2,LinP_3)
  Tab_13 = column(Espacio,checkbox_pico_valle)
  #herramienta filtros
  #PRINCIPALES
  tab1 = TabPanel(child=p, title="GRAFICA")#Usar child=... porque TabPanel todavía acepta esa sintaxis
  tab2 = TabPanel(child=contenido_tabla, title="TABLA")
  tab3 = TabPanel(child=Tab_12, title="FILTROS")
  tab3_3 = TabPanel(child=Tab_13, title="Control de calidad")
  tab4 = TabPanel(child=col_controles, title="EDICION DEL LIENZO")


  #tab4 = TabPanel(child=checkbox, title="Cargar")

  #PRINCIPALES
  tabs = Tabs(tabs=[tab1])
  #HERRAMIENTAS

  #ESTILOS TABS
  tabs44 = Tabs(tabs=[tab3,tab2,tab4,tab3_3])

  general = row(tabs,tabs44)

  script, div = components(general)

  recursos = CDN.render()
  display(HTML(recursos + div + script))

  with Resultado_toga:
    clear_output()
    scripts = [
          "/content/PruebaconGoogleColab/ET1_ proyecto 1/GENERAL/SCRIPTS -- de practica/06_comprobar_datos_toga.py",
          "/content/PruebaconGoogleColab/ET1_ proyecto 1/GENERAL/SCRIPTS -- de practica/11_control_calidad_automatizado.py"
      ]
    for file_N in archivos_cargados:
      for script in scripts:
          comando = ['python3', script, file_N]
          print(f"\n--- Ejecutando: {script} ---")
          resultado = subprocess.run(comando, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
          print(resultado.stdout)
          if resultado.stderr:
              print("⚠️ Error:", resultado.stderr)
              print("Verifica que ya hayas llamado el repositorio")
              print("que contiene los scripts")
      print("")
      print("#######################################################################################")
      print("")
  with Archivo_toga:
    clear_output()
    file_N = []
    for file_N in archivos_cargados:
      print("archivos cargados",file_N)
      matriz_toga = pd.read_csv(file_N,sep=r'\s+',names =["StationID", "StationName", "Date", "D1", "D2", "D3","D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"], engine = 'python', skiprows = 1, na_values = "9999")
      #print("Matriz toga",matriz_toga.head())
      # version mejorada de google colab
      #display(matriz_toga.head()) # podemos ponerle valore al head(10) conforme a las filas que queremos ver
      display(matriz_toga)  # Muestra todo el DataFrame completo

  # Crear la figura
#[11]==================== CARGADOR DE ARCHIVO =================================
# Inicializamos selector visible
contenedor_selector = widgets.Output()
Contenedor_salida= widgets.Output()
#tabs de
Archivo_toga = widgets.Output()
#================================================================================
#[1]=================  CARGAR ARCHIVOS  ==========================
clear_output()
ruta_archivo=[]
#revisamos si no existe esta ruta
if not os.path.exists('/content/drive'):

  #usamos el modulo drive que conecta a la funcion mount, para asi montar nuestra ruta
  drive.mount('/content/drive')

#guardamos nuestra ruta base en ruta_
# ORIGINAL ------ ruta_drive = '/content/drive/MyDrive'
ruta_drive = '/content'

#mas adelante podemos modificaremos su contenido de forma dinamica
ruta_actual = [ruta_drive]
#[2]=================  CARGAR ARCHIVOS  ==========================
# nuestra lista general de archivos
rutas_seleccionadas = []

#creamos nuestros contenedores para mostrar resultados
#con Output() creamos una area de salida interactiva, para mostrar errores y cualquier salida
#esto lo mostraremos mas adelante
salida_pc = widgets.Output()
salida_drive = widgets.Output()
salida_resultado = widgets.Output()
salida_seleccionado=widgets.Output()
output = widgets.Output()
#creamos dos boton que mostara un texto personalizado con estilo 'info'
#esto lo usaremos mas adelante
boton_pc = widgets.Button(description="Desde PC",button_style='info')
boton_drive = widgets.Button(description="Navegar en Drive",button_style='primary')
#[3]=================  CARGAR ARCHIVOS  ==========================
#creamos nuestra funcion para mostrar el contenido seleccionado desde drive
def mostrar_contenido_drive():
  #usamos nuestro contenedor para mostrar el contenido que continua
  with salida_drive:
    #eliminamos todo lo anterior
    clear_output()
    with salida_seleccionado:
      #print("Has seleccionado",rutas_seleccionadas)
      clear_output()
      print("Has seleccionado")
      for iteracion in rutas_seleccionadas:
        print("-",iteracion)
      print("========================================")
    #obtenemos el primer elemento de la lista
    actual = ruta_actual[0]
    #listamos todas las carpetas(al igual que archivos) y las guardamos en archivos
    archivos = os.listdir(actual)


    #para cada elemento f  en la lista de archivos, si f es uan carpeta dentro de la ruta actual
    #agregar f a la lista de carpetas
    carpetas = sorted([f for f in archivos if os.path.isdir(os.path.join(actual,f))])

    #para cada elemento en la lista de archivos, si es un archivo agregalos a unalista
    #sorted nos ayuda a optener un alfabeticamente la lista interna y esa lista lo guardamos en ficheros
    ficheros = sorted([f for f in archivos if os.path.isfile(os.path.join(actual,f))])

    #cremos un widget de seleccion multiple
    #esto mostrara todos los archivos y podemos seleccionar varios
    selector = widgets.SelectMultiple(
        #unir las listas de carpetas y ficheros
        options = carpetas + ficheros,
        #agregamos una etiqueta de contenido
        description = 'Contenido:',
        #mostrar hasta 10 filas visibles a la vez
        rows = 10,
        #ajustamos el ancho del widget al 80%
        layout = widgets.Layout(width='80')
    )
    #creamos nuestros botones
    boton_entrar = widgets.Button(description="<Entrar a carpeta>", button_style='primary')
    boton_atras = widgets.Button(description="<--Ir atras",button_style='warning')
    boton_seleccionar = widgets.Button(description = ">> Seleccionar archivo <<", button_style='success')

    #creamos nuestra funcion al hacer click en una carpeta
    def entrar_click(b):
      # leemos lo que el usuario ya ha seleccionado desde el widget en selector
      #convertimos esa seleccion en unalista y lo guardamos en la variable seleccion
      seleccion = list(selector.value)
      # si la cantidad de elementos seleccionados es = a 1, hacer lo siguiente
      if len(seleccion) ==1:
        #le asignamos a la variable nombre el primer elemento seleccionado
        nombre= seleccion[0]
        #recordemos que el ruta actual esta como  '/content/drive/MyDrive'
        #entonces le pasamos el nombre del archivo seleccionado
        #nueva ruta = tiene ahora la ruta del archivo seleccionado
        nueva_ruta = os.path.join(actual,nombre)

        #si la ruta guardada en 'nueva_ruta'es una carpeta
        #nueva_ruta = es igual al archivo seleccionado
        if os.path.isdir(nueva_ruta):
          # le pasamos la nueva ruta el primer elemento en posicion 0 de la lista de ruta actual
          ruta_actual[0] = nueva_ruta
          # y vamos de nuevo a mostrar el contenido de drive
          mostrar_contenido_drive()
        # y si no es asi pues
        else:
          #mostramos el widget de salida de resultado
          # donde limpiamos y imprimimos un mensaje
          with salida_resultado:
            clear_output()
            print("Solo puedes seleccionar una carpeta a la vez")
      #si no se ha seleccionado uno pues asemos lo siguiente
      else:
        #donde mostramos el siguiente mensaje de salida resultado
        with salida_resultado:
          clear_output()
          print("Solo puedes seleccionar una carpeta para entrar")

    # esto es para cuando hemos seleccionado nuestro boton de ir atras
    # a la funcion le pasamos el parametro b como representacion del boton presionado
    def atras_click(b):
      #preguntamos si ruta actual es diferente a ruta drive
      # si si estonces la condicion se cumple
      if ruta_actual[0] != ruta_drive:
        #cambiamos la ruta actual por {la carpeta anterior}
        ruta_actual[0] = os.path.dirname(ruta_actual[0])
        #de nuevo volvemos a iniciar
        mostrar_contenido_drive()

    # ahora con el boton de seleccionar el archivos
    def seleccionar_click(b):
      #leemos los elementos seleccioados  y con el widget de selector
      # ahora value devuelve los elementos seleccionado de forma tupla
      #('carpeta','archivo1.txt') pero con list lo convertimos en una lista
      seleccionados = list(selector.value)
      #inicializamos una variable
      agregados = 0

      #con la variable nombre recorremos la lista de elementos
      # ejemplo nombre se asigna ra el nombre de cada archivo acumulado en seleccionados
      #nombre = 'archivo.dat'
      for nombre in seleccionados:
        #unimos la ruta actual con nombre que se esta iterando
        #actual es la ruta y en ruta completa se almacenara la ruta general
        ruta_completa = os.path.join(actual,nombre)
        #si existe el archivo en la ruta completa
        if os.path.isfile(ruta_completa):
          #si es asi entonces verificamos si ruta completa no esta en nuestra lista de rutas seleccionadas
          if ruta_completa not in rutas_seleccionadas:
            #si esto se comple
            #agregamos la ruta_completa asta el final de rutas seleccionadas
            rutas_seleccionadas.append(ruta_completa)
            #y incrementamos a agregados
            agregados =+1
          else:
            print("")
            print(f"==> Archivo duplicado (contenido igual){os.path.basename(ruta_completa)} <==")
      #mostramos_rutas() es donde tenemos una funcion que muestra las ruta recien agregadas
      mostrar_rutas()
      #ademas verificamos si agregados fue incrementado
      # es para saber si hubo archivos añadidos
      if agregados == 0:
        print("Ningun archivo nuevo fue agregado")
    #cuando presionemos el boton vamos a entrar a ejecutar sus funciones
    boton_entrar.on_click(entrar_click)
    boton_atras.on_click(atras_click)
    boton_seleccionar.on_click(seleccionar_click)

    #mostramos una caja vertical
    display(widgets.VBox([
        #usamos un texto html que dice ruta actual en negrita
        widgets.HTML(f"<b>Ruta actual:</b>{actual}"),
        #el selector de opcion
        selector,
        #una fila horizontal con los botones atras, entrar y seleccionar
        widgets.HBox([boton_atras,boton_entrar,boton_seleccionar])
    ]))
# un conjunto vacio de coleciones de elementos unico y no ordenados
hashes_archivos = set()

def obtener_hash(archivo_path):
  hash_md5 = hashlib.md5()
  with open(archivo_path, "rb") as f:
    for chunk in iter(lambda:f.read(4096),b""):
      hash_md5.update(chunk)
  return hash_md5.hexdigest()

def seleccionar_desde_pc(b=None):
  with salida_pc:
    clear_output()
    print("Sube un archivo desde tu PC:")
    #print("Has seleccionado",rutas_seleccionadas)
    #print("Rutas Guardadas",ruta_archivo)
    upload = files.upload()

    for nombre in upload.keys():
      ruta_local = os.path.abspath(nombre)
      hash_archivo = obtener_hash(ruta_local)
      if hash_archivo not in hashes_archivos:
        rutas_seleccionadas.append(ruta_local)
        hashes_archivos.add(hash_archivo)
        print("")
        print(f"==> Archivo añadido:{os.path.basename(ruta_local)} <==")
      else:
        print("")
        print(f"==> Archivo duplicado (contenido igual){os.path.basename(ruta_local)} <==")
        print("")
    mostrar_rutas()

def mostrar_rutas():
    with salida_resultado:
        clear_output()
        if rutas_seleccionadas:
            print("")
            print("Archivos seleccionados")
            print("")
            for i, ruta in enumerate(rutas_seleccionadas, 1):
                print(f"{i}. {ruta}")
                ruta_archivo.append(ruta)
        else:
            print("No se ha seleccionado ningún archivo aún")
    print("Has seleccionado",rutas_seleccionadas)
    with Contenedor_salida:
      clear_output()
      actualizar_selector()
def activar_pc(b):
  salida_drive.clear_output()
  salida_resultado.clear_output()
  ruta_actual[0]=ruta_drive
  seleccionar_desde_pc(None)

def activar_drive(b):
  salida_pc.clear_output()
  salida_resultado.clear_output()
  ruta_actual[0]=ruta_drive
  mostrar_contenido_drive()

#[1]==================== CARGADOR DE ARCHIVO =================================
# Archivos "pre-cargados"
#aqui como es una lista usamos un for
nombre=[]
archivos_disponibles = rutas_seleccionadas

from os.path import basename

# Crear opciones como lista de tuplas (nombre_visible, ruta_real)
opciones = [(os.path.basename(r), r) for r in archivos_disponibles]

#[2]==================== CARGADOR DE ARCHIVO =================================
# Texto de estado en pestaña 1
texto_funcion = widgets.Label(value="Función desactivada")
#[3]==================== CARGADOR DE ARCHIVO =================================
# Checkbox para activar función (que cambia tipo de selección)
checkbox_activar_funcion = widgets.Checkbox(
    value=False,
    description="Selector multiple"
)
#[4]==================== CARGADOR DE ARCHIVO =================================
# Selector de archivos (se cambiará dinámicamente entre Select y SelectMultiple)
with Contenedor_salida:
    clear_output()
    selector_archivos_single = widgets.Select(
        options=opciones,
        description="Archivo:",
        rows=6
    )
    selector_archivos_multiple = widgets.SelectMultiple(
        options=opciones,
        description="Archivos:",
        rows=6
    )
#[5]==================== CARGADOR DE ARCHIVO =================================
# Inicialmente usamos selector simple
selector_archivos_actual = selector_archivos_single
#[6]==================== CARGADOR DE ARCHIVO =================================
# Botón procesar
boton_procesar = widgets.Button(description="Procesar")

#[7]==================== CARGADOR DE ARCHIVO =================================
# Output detalles en pestaña 2
output_detalle = widgets.Output()
Resultado_toga = widgets.Output()
#[A1]==================== Grafica Dividida =================================
Graficar = widgets.Output()
Captura_tabla_general = widgets.Output()
#[8]==================== CARGADOR DE ARCHIVO =================================
# Lista global de archivos procesados
archivos_procesados = []

#[9]==================== CARGADOR DE ARCHIVO =================================
def actualizar_selector():
    global selector_archivos_actual

    if checkbox_activar_funcion.value:
        # Función activada: usamos SelectMultiple para permitir seleccionar varios
        selector_archivos_actual = selector_archivos_multiple
        #nombre = [os.path.basename(ruta) for ruta in rutas_seleccionadas]
        #selector_archivos_actual.options = nombre
        selector_archivos_actual.options = archivos_disponibles
    else:
        # Función desactivada: selector simple para seleccionar sólo uno
        selector_archivos_actual = selector_archivos_single

        selector_archivos_actual.options = archivos_disponibles
#[10]==================== CARGADOR DE ARCHIVO =================================
def cambiar_selector(change=None):
    # Cuando cambia el checkbox, actualizamos el selector visible y texto
    actualizar_selector()
    # Actualizamos texto
    if checkbox_activar_funcion.value:
        texto_funcion.value = "Función activada"
    else:
        texto_funcion.value = "Función desactivada"
    # Reemplazamos en el layout
    # Quitamos widget viejo y ponemos nuevo
    with contenedor_selector:
        clear_output()
        display(selector_archivos_actual)

checkbox_activar_funcion.observe(cambiar_selector, names='value')


with contenedor_selector:
    display(selector_archivos_actual)
#[A3]==================== Grafica Dividida =================================
dfs = []
archivos_cargados = []

#[12]==================== CARGADOR DE ARCHIVO =================================
def procesar_click(b):
    global archivos_procesados,dfs, archivos_cargados, tabla_local, tablas,linea1 # <- importante
    # Tomamos seleccionados según el modo
    if checkbox_activar_funcion.value:
        seleccionados = list(selector_archivos_actual.value)
    else:
        seleccionados = [selector_archivos_actual.value] if selector_archivos_actual.value else []
    archivos_procesados = seleccionados

    with output_detalle:
        clear_output()
        print("")
        print("Archivos procesados:")
        for f in archivos_procesados:
            print(f"- {f}")
        rango = len(archivos_procesados)
        if rango >= 3:
          print("SELECCIONO",rango)
          print("⚠️Tenga cuidado al usar esta funcion puede que se relentice la interaccion ⚠️")

        #[A2]==================== Grafica Dividida =================================
        #LOGICA DE GRAFICA
        with Graficar:
          clear_output()

          if not archivos_procesados:
            print("No se cargaron archivos.")
            return


          # Ya NO hagas dfs = [] aquí, solo modifica la global si quieres reiniciar
          # Si quieres reiniciar los datos, hazlo aquí:

          dfs.clear()
          archivos_cargados.clear()
          tabla_local.clear()
          tablas.clear()
          linea1.clear()
          for archivo in archivos_procesados:
            matriz_toga = pd.read_csv(archivo, sep=r'\s+', names=["StationID", "StationName", "Date",
                                "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"],
                                engine='python', skiprows=1, na_values="9999")

            lista_fecha = []
            lista_datos = []

            for filas_dat in matriz_toga.index:
                fecha = str(matriz_toga["Date"][filas_dat])
                if fecha[8:9] == "1":
                    fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]))
                else:
                    fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]), 12)

                for dat in ["D1", "D2", "D3", "D4",  "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"]:
                    lista_fecha.append(fecha_inicial)
                    lista_datos.append(matriz_toga[dat][filas_dat])
                    fecha_inicial += datetime.timedelta(hours=1)

            df = pd.DataFrame({"Date": lista_fecha, "Promedio": lista_datos,"Archivo":archivo})
            dfs.append(df)
            archivos_cargados.append(archivo)


          actualizar_vista()


#[13]==================== CARGADOR DE ARCHIVO =================================
#definimos auque esten globales por que nos aparece un error
boton_procesar.on_click(procesar_click)
archivos_cargados = []
tabla_local = []
tablas = []
linea1=[]
#[14]==================== CARGADOR DE ARCHIVO =================================
# Layout pestaña 1
tab1 = widgets.VBox([
    texto_funcion,
    checkbox_activar_funcion,
    contenedor_selector,
    boton_procesar,
])

# Layout pestaña 2
tab2_resultado = widgets.VBox([
    output_detalle
])

#[1]=================  Cuadros  ==========================
output_notebook()
#[2]=================  Cuadros  ==========================
style_box = {
    'border': '5px solid #2D6FA4',
    'padding': '10px',
    'margin': '1.5px',
    'border-radius': '8px',
    'background-color': '#10283B',  #
    'width': '570px',
    'height': '280px',
    'overflow': 'auto',
}
#[3]=================  Cuadros  ==========================
#todo el contenido del cuadrito
def make_box(title, content_widget):
    label = widgets.HTML(value=f"<b>{title}</b>")
    # VBox para apilar el título y el contenido
    box = widgets.VBox([label, content_widget])
    #[4]=================  Cuadros  ==========================
    box.layout = widgets.Layout(
        border=style_box['border'],
        padding=style_box['padding'],
        margin=style_box['margin'],
        border_radius=style_box['border-radius'],
        background_color=style_box['background-color'],
        width=style_box['width'],
        height=style_box['height'],
        overflow=style_box['overflow'],
    )
    return box
#[4]=================  CARGAR ARCHIVOS  ==========================
boton_pc.on_click(activar_pc)
boton_drive.on_click(activar_drive)
salida_resultado.layout = widgets.Layout(margin='0 10px 0 0')  # margen derecho
salida_drive.layout = widgets.Layout(margin='0 10px 0 10px')   # márgenes laterales
salida_pc.layout = widgets.Layout(margin='0 0 0 10px')          # margen izquierdo
salida_seleccionado.layout = widgets.Layout(margin='0 0 0 10px')          # margen izquierdo
#[5]=================  CARGAR ARCHIVOS  ==========================
# Agruparlos en una fila (HBox) y centrar
hbox_botones = widgets.HBox(
      [boton_pc, boton_drive],
      layout=widgets.Layout(justify_content='center')
  )
# Diseño a los salidas
hbox_botones2 = widgets.HBox(
      [salida_drive,salida_pc],
      layout=widgets.Layout(justify_content='center')
)
#[5]=================  Cuadros  ==========================
# Ejemplo de contenido: widget con texto largo para activar scroll
resultado_general = widgets.VBox(
    [salida_seleccionado,salida_resultado],
    layout=widgets.Layout(justify_content='center')

)
#[6]=================  Cuadros  ==========================
# Otro contenido: lista de botones (para mostrar ejemplo variado)
#long_text2 = widgets.HTML(
#    value="<br>".join([f"Línea {i}" for i in range(50)])
#)
#[7]=================  Cuadros  ==========================
# Crear cajas con contenido variado
box1 = make_box("Cargar Archivo",hbox_botones)
box1.layout.width ='2000px'
box1.layout.height ='100px'
box1.layout.justify_content = 'center'   # centra horizontal
box1.layout.align_items = 'center'
box2 = make_box("Resultado",resultado_general)
box2.layout.width ='1000px'
box2.layout.height ='500px'
box3 = make_box("Archivos cargados",hbox_botones2)
box3.layout.width ='1000px'
box3.layout.height ='500px'

#[8]=================  Cuadros  ==========================
# Organizar cajas en grid visual con HBox y VBox
fila = widgets.HBox([box1])
fila1 = widgets.HBox([box3,box2])
dashboard = widgets.VBox([fila,fila1])

# [1]=================  Ventana  ==========================
#VENTANA 2---- contenido de ventana 2
# Contenido para pestañas internas de la pestaña anidada

inner_tab1 = widgets.HBox([
    tab1,tab2_resultado
    ])
#inner_tab2 = widgets.Label(value="Filtros")
#inner_tab3 = widgets.Label(value="EDicion grafica")
# [2]=================  Ventana  ==========================
#VENTANA 2----- Pestañas de la ventana
# Crear el widget de pestañas internas (anidadas)

#inner_tabs = widgets.Tab(children=[inner_tab1, inner_tab2,inner_tab3])# [la llamada]
#inner_tabs.set_title(0, 'Cargar archivos') # === El nombre ====
#inner_tabs.set_title(1, 'Filtro')
#inner_tabs.set_title(2, 'Edicion')

#VENTANA 1 ---- padre
# Contenido para pestañas externas (ventana principal)
# [3]=================  Ventana  ==========================
title = widgets.HTML(
    "<h1 style='text-align:center;'>Panel para seleccionar archivos que ya subiste ---- Presiona aquí para desplegar y esconder el contenido</h1>"
)
contenido = widgets.VBox([title,inner_tab1])
accordion = widgets.Accordion(children=[contenido])
accordion.set_title(0,">>>")
tab1_content = widgets.VBox([
    dashboard,
])
#VENTANA 1 --- sus demas comparñeros
# [3]=================  Ventana  ==========================
tab2_content = widgets.VBox([
    #Ventana 2
    #aqui va la grafica
    #inner_tabs,  # Aquí va la ventana anidada dentro de la pestaña 1

    accordion,
    Graficar
    ])
tab3_content = widgets.VBox([
    widgets.Label(value="Aqui veras el el analisis de los archivos"),#contenido texto
    Resultado_toga
])
tab4_content = widgets.VBox([
    widgets.Label(value="Aqui veras el el analisis de los archivos"),#contenido texto
    Archivo_toga
])
tab5_content = widgets.VBox([
    widgets.Label(value="Aqui veras el el analisis de los archivos"),#contenido texto

])
tab6_content = widgets.VBox([
    widgets.Label(value="Aqui veras el el analisis de los archivos"),#contenido texto

])
# Crear el widget de pestañas principal
# [2]=================  Ventana  ==========================
tabs = widgets.Tab(children=[tab1_content, tab2_content, tab3_content,tab4_content,tab5_content,tab6_content])# [la llamada]
tabs.set_title(0, 'INICIO') # ===El nombre====
tabs.set_title(1, 'GRAFICA')
tabs.set_title(2, 'Salida')
tabs.set_title(3, 'Archivo toga') # ===El nombre====
tabs.set_title(4, 'Archivos graficados')
tabs.set_title(5, 'Salida')
############################################TITULO MAREOGRAFICO
html_con_estilo = f"""
<div style="min-height:100px; border: 0px solid #ccc; padding:10px;">
    <img src="https://i.postimg.cc/BvXVBNvX/imagen-mareografico-tema.png"
         alt="Ejemplo"
         style="max-width:100%; height:200px; display:block; margin:auto;">
</div>
"""

display(HTML(html_con_estilo))
############################################
# Mostrar la ventana principal
display(tabs)
vista = f"""
  <div style="min-height:300px; border: 1px solid #ccc; padding:10px;background-color:#2D6FA4">
  </div>
  """
display(HTML(vista))


Mounted at /content/drive
